# CORD-19-collect-scopus-data

In general, this jupyter notebook is designated to collect additional data via scopus to enbroaden the CORD19 dataset: 
https://datadryad.org/stash/dataset/doi:10.5061/dryad.vmcvdncs0

First, relevant packages must be imported to the Notebook.

In [1]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

Get the data and save it to a variable.

In [2]:
CORD19_CSV = pd.read_csv('../data/cord-19/CORD19_software_mentions.csv')

Check the length of the column containing doi's.

In [3]:
len(CORD19_CSV['doi'])

77448

Display the column doi to see if there are inconsistencies such as NaN's

In [4]:
doi = CORD19_CSV['doi']
doi

0                                 NaN
1          10.1016/j.regg.2021.01.002
2           10.1016/j.rec.2020.08.002
3        10.1016/j.vetmic.2006.11.026
4                   10.3390/v12080849
                     ...             
77443      10.1007/s11229-020-02869-9
77444                             NaN
77445     10.1101/2020.05.13.20100206
77446      10.1007/s42991-020-00052-8
77447     10.1101/2020.09.14.20194670
Name: doi, Length: 77448, dtype: object

Create a series with solely unique values and neglect NaN's. It is important to sort the unique values. Otherwise, the method is creating different results after each restart of the notebook. 

In [5]:
doi_counted = doi.value_counts().sort_index(ascending=True)
doi_counted

10.1001/jamainternmed.2020.1369       1
10.1001/jamanetworkopen.2020.16382    1
10.1001/jamanetworkopen.2020.17521    1
10.1001/jamanetworkopen.2020.20485    1
10.1001/jamanetworkopen.2020.24984    1
                                     ..
10.9745/ghsp-d-20-00115               1
10.9745/ghsp-d-20-00171               1
10.9745/ghsp-d-20-00218               1
10.9758/cpn.2020.18.4.607             1
10.9781/ijimai.2020.02.002            1
Name: doi, Length: 74302, dtype: int64

The following function determines the requested information from the Scopus API. (https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4)

In [6]:
#Adapted from https://github.com/ElsevierDev/elsapy/blob/master/exampleProg.py
def fetch_scopus_api(client, doi):
    """obtain additional paper information from scopus by doi
    """
    doc_srch = ElsSearch("DOI("+doi+")",'scopus')
    doc_srch.execute(client, get_all = True)
    #print ("doc_srch has", len(doc_srch.results), "results.")
    #print(doc_srch.results)
    try:
        scopus_id=doc_srch.results[0]["dc:identifier"].split(":")[1]
        scp_doc = AbsDoc(scp_id = scopus_id)
        if scp_doc.read(client):
            # print ("scp_doc.title: ", scp_doc.title)
            scp_doc.write()   
        else:
            print ("Read document failed.")
        # print(scp_doc.data["affiliation"])
        return scp_doc.data
    except:
        return None

Thusly, the configuration file is set up and contains an APIkey. Further information: https://github.com/ElsevierDev/elsapy/blob/master/CONFIG.md

In [7]:
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

Moreover, the client is initialized with the API-Key.

In [8]:
client = ElsClient(config['apikey'])

For demonstation purposes, the following cells shows which data is returned by the Scopus API. 

In [9]:
return_example = fetch_scopus_api(client, '10.1016/j.dsx.2020.04.012')
print(json.dumps(return_example, indent=2))

{
  "affiliation": [
    {
      "affiliation-city": "New Delhi",
      "affilname": "Jamia Hamdard",
      "affiliation-country": "India"
    },
    {
      "affiliation-city": "New Delhi",
      "affilname": "Jamia Millia Islamia",
      "affiliation-country": "India"
    },
    {
      "affiliation-city": "New Delhi",
      "affilname": "Indraprastha Apollo Hospitals",
      "affiliation-country": "India"
    }
  ],
  "coredata": {
    "srctype": "j",
    "eid": "2-s2.0-85083171050",
    "pubmed-id": "32305024",
    "prism:coverDate": "2020-07-01",
    "prism:aggregationType": "Journal",
    "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85083171050",
    "dc:creator": {
      "author": [
        {
          "ce:given-name": "Raju",
          "preferred-name": {
            "ce:given-name": "Raju",
            "ce:initials": "R.",
            "ce:surname": "Vaishya",
            "ce:indexed-name": "Vaishya R."
          },
          "@seq": "1",
          "ce:init

Based on the returned data, further analysis is conductable. Therefore, two notebooks are created to analyse data linked to: 
<ul>
  <li>affiliation</li>
  <li>coredata</li>
</ul>    

Thusly, the already fetched coredata and affiliation are read and combined to a DataFrame for further processing.

In [10]:
df_current_extra_info = pd.DataFrame()
try:
    read_affiliation = pd.read_pickle('extra_info_affiliation_CS.pkl')
    read_coredata = pd.read_pickle('extra_info_coredata_CS.pkl')
    df_current_extra_info['affiliation'] = read_affiliation
    df_current_extra_info['coredata'] = read_coredata
    df_current_extra_info
except:
    print("The DataFrame is empty")
    #if the dataframe is not empty set the variable to show the dataframe

The length of the DataFrame containing the current information is assigned to a variable to be used for further processing. 
Therefore, the length will be used within a while loop as a starting index. 

In [11]:
len_df_current_extra_info = len(df_current_extra_info)
len_df_current_extra_info

71829

In [12]:
df_current_extra_info

,affiliation,coredata
0,"[{'affiliation-city': 'Palo Alto', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083266658', ..."
1,"[{'affiliation-city': 'Seattle', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
2,"[{'affiliation-city': 'Cambridge', 'affilname'...","{'srctype': 'j', 'prism:issueIdentifier': '8',..."
3,"[{'affiliation-city': 'Madison', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '9',..."
4,"[{'affiliation-city': 'Los Angeles', 'affilnam...","{'srctype': 'j', 'prism:issueIdentifier': '11'..."
...,...,...
71824,"{'affiliation-city': 'Pathum Thani', 'affilnam...","{'srctype': 'j', 'prism:issueIdentifier': '3',..."
71825,"[{'affiliation-city': 'Pittsburgh', 'affilname...","{'srctype': 'j', 'prism:issueIdentifier': '3',..."
71826,"[{'affiliation-city': 'Lyon', 'affilname': 'Ce...","{'srctype': 'j', 'prism:issueIdentifier': '3',..."
71827,"[{'affiliation-city': 'Beijing', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '4',..."


In [13]:
def contains_only_None(dic):
    """
    This functions inspects an dictionary and returns True if it solely contains None values
    """
    return len(dic) == sum(value == None for value in dict_new_extra_info.values())

In [14]:
def append_fetched_data_to_df(df_current_extra_info, dic):
    """
    This function appends or inserts newly fetched data to the DataFrame containing scopus data.
    """
    #df_current_extra_info -> holding the latest data, new data needs to be appended to it, 
    #df_newly_fetched_transposed -> holdy newly fetched data, needs to be inserted or fetched
    
    if contains_only_None(dic):
        placeholder_entries = pd.DataFrame(np.empty((len(dict_new_extra_info),2),dtype=object),columns=['affiliation','coredata'], index=dict_new_extra_info.keys())
        df_newly_fetched_transposed = placeholder_entries
        print(placeholder_entries)
    else:
        #Prior appending, the dictionary is transformed to a DataFrame
        df_newly_fetched = pd.DataFrame(dic)
        #For readability, the DataFrame is transposed
        df_newly_fetched_transposed = df_newly_fetched.T
        print(df_newly_fetched_transposed)
    
    #Insert newly fetched rows which were previously not successful appended
    for index, row in df_newly_fetched_transposed.iterrows():
        #insert to current extra info DataFrame because the row is existent
        if index in df_current_extra_info.index and row.affiliation is not None:
            df_current_extra_info.loc[index] = row
        #append to current extra info DataFrame because the row is new     
        if index not in df_current_extra_info.index:
            df_current_extra_info = df_current_extra_info.append(row, ignore_index=True)
            
    #returning DataFrame with inserted and replaced rows. 
    return df_current_extra_info

Both Dataframes columns are stored each to an object. The series objects are stored to each to a pkl-file which is not exceeding the size of 100MB allowing GitHub uploads.

In [15]:
def store_df_columns(df):
    ser_affiliation = df['affiliation']
    ser_coredata = df['coredata']
    ser_affiliation.to_pickle('extra_info_affiliation_CS.pkl')
    ser_coredata.to_pickle('extra_info_coredata_CS.pkl')
    return ser_affiliation, ser_coredata

In [16]:
# placeholder_entries = pd.DataFrame(np.empty((4,2),dtype=object),columns=['affiliation','coredata'])

In [17]:
# placeholder_entries

Subsequently, the fetched scopus data is stored within a dictionary. Besides, the print function is used to show the state of the process by displaying the latest fetched information. 

In [ ]:
%%time
dict_new_extra_info = dict()
len_dois = len(doi_counted)
def trigger_fetching():
    threshold = 0 
    i = len_df_current_extra_info
    while i < len_dois: #-> upto modified, normally len_dois
        dict_new_extra_info[i] = fetch_scopus_api(client, doi_counted.index[i])
        print("Position fetched: " + str(i) + " -> " +  doi_counted.index[i])
        i = i + 1 
        threshold = threshold + 1
        if threshold > 99:
            df_combined_extra_info = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info)
            stored_series = store_df_columns(df_combined_extra_info)
            threshold = 0
            print("batch saved")
trigger_fetching()

Position fetched: 71829 -> 10.3390/v11040326
Position fetched: 71830 -> 10.3390/v11040328
Position fetched: 71831 -> 10.3390/v11040356
Position fetched: 71832 -> 10.3390/v11040374
Position fetched: 71833 -> 10.3390/v11040377
Position fetched: 71834 -> 10.3390/v11040379
Position fetched: 71835 -> 10.3390/v11040381
Position fetched: 71836 -> 10.3390/v11040382
Position fetched: 71837 -> 10.3390/v11040385
Position fetched: 71838 -> 10.3390/v11050394
Position fetched: 71839 -> 10.3390/v11050398
Position fetched: 71840 -> 10.3390/v11050403
Position fetched: 71841 -> 10.3390/v11050410
Position fetched: 71842 -> 10.3390/v11050420
Position fetched: 71843 -> 10.3390/v11050423
Position fetched: 71844 -> 10.3390/v11050427
Position fetched: 71845 -> 10.3390/v11050444
Position fetched: 71846 -> 10.3390/v11050448
Position fetched: 71847 -> 10.3390/v11050471
Position fetched: 71848 -> 10.3390/v11050477
Position fetched: 71849 -> 10.3390/v11060489
Position fetched: 71850 -> 10.3390/v11060497
Position f

Position fetched: 71979 -> 10.3390/v12040449
Position fetched: 71980 -> 10.3390/v12040473
Position fetched: 71981 -> 10.3390/v12050497
Position fetched: 71982 -> 10.3390/v12050498
Position fetched: 71983 -> 10.3390/v12050502
Position fetched: 71984 -> 10.3390/v12050506
Position fetched: 71985 -> 10.3390/v12050513
Position fetched: 71986 -> 10.3390/v12050514
Position fetched: 71987 -> 10.3390/v12050518
Position fetched: 71988 -> 10.3390/v12050524
Position fetched: 71989 -> 10.3390/v12050526
Position fetched: 71990 -> 10.3390/v12050530
Position fetched: 71991 -> 10.3390/v12050534
Position fetched: 71992 -> 10.3390/v12050536
Position fetched: 71993 -> 10.3390/v12050539
Position fetched: 71994 -> 10.3390/v12050540
Position fetched: 71995 -> 10.3390/v12050543
Position fetched: 71996 -> 10.3390/v12050553
Position fetched: 71997 -> 10.3390/v12050560
Position fetched: 71998 -> 10.3390/v12050562
Position fetched: 71999 -> 10.3390/v12050576
Position fetched: 72000 -> 10.3390/v12050578
Position f

                                             affiliation  \
71829  [{'affiliation-city': 'Edmonton', 'affilname':...   
71830  [{'affiliation-city': 'Krakow', 'affilname': '...   
71831  [{'affiliation-city': 'Singapore City', 'affil...   
71832  [{'affiliation-city': 'Geneva', 'affilname': '...   
71833  [{'affiliation-city': 'Bern', 'affilname': 'Un...   
...                                                  ...   
72124  [{'affiliation-city': 'Saint-Laurent', 'affiln...   
72125  [{'affiliation-city': 'Riyadh', 'affilname': '...   
72126  [{'affiliation-city': 'Taipei', 'affilname': '...   
72127  {'affiliation-city': 'Poznan', 'affilname': 'I...   
72128  [{'affiliation-city': 'Cambridge', 'affilname'...   

                                                coredata  
71829  {'srctype': 'j', 'prism:issueIdentifier': '4',...  
71830  {'srctype': 'j', 'prism:issueIdentifier': '4',...  
71831  {'srctype': 'j', 'prism:issueIdentifier': '4',...  
71832  {'srctype': 'j', 'prism:issueIdentif

Position fetched: 72247 -> 10.3390/v5020550
Position fetched: 72248 -> 10.3390/v5071607
Position fetched: 72249 -> 10.3390/v5081991
Position fetched: 72250 -> 10.3390/v5102375
Position fetched: 72251 -> 10.3390/v5112679
Position fetched: 72252 -> 10.3390/v5123007
Position fetched: 72253 -> 10.3390/v6010137
Position fetched: 72254 -> 10.3390/v6020516
Position fetched: 72255 -> 10.3390/v6031004
Position fetched: 72256 -> 10.3390/v6031037
Position fetched: 72257 -> 10.3390/v6031253
Position fetched: 72258 -> 10.3390/v6031317
Position fetched: 72259 -> 10.3390/v6041654
Position fetched: 72260 -> 10.3390/v6051876
Position fetched: 72261 -> 10.3390/v6051897
Position fetched: 72262 -> 10.3390/v6052084
Position fetched: 72263 -> 10.3390/v6082938
Position fetched: 72264 -> 10.3390/v6093386
Position fetched: 72265 -> 10.3390/v6124880
Position fetched: 72266 -> 10.3390/v6125047
Position fetched: 72267 -> 10.3390/v7010239
Position fetched: 72268 -> 10.3390/v7010352
Position fetched: 72269 -> 10.33

Position fetched: 72393 -> 10.3390/vaccines8030364
Position fetched: 72394 -> 10.3390/vaccines8030378
Position fetched: 72395 -> 10.3390/vaccines8030379
Position fetched: 72396 -> 10.3390/vaccines8030386
Position fetched: 72397 -> 10.3390/vaccines8030389
Position fetched: 72398 -> 10.3390/vaccines8030397
Position fetched: 72399 -> 10.3390/vaccines8030408
Position fetched: 72400 -> 10.3390/vaccines8030423
Position fetched: 72401 -> 10.3390/vaccines8030443
Position fetched: 72402 -> 10.3390/vaccines8030444
Position fetched: 72403 -> 10.3390/vaccines8030462
Position fetched: 72404 -> 10.3390/vaccines8030468
Position fetched: 72405 -> 10.3390/vaccines8030471
Position fetched: 72406 -> 10.3390/vaccines8030472
Position fetched: 72407 -> 10.3390/vaccines8030474
Position fetched: 72408 -> 10.3390/vaccines8030480
Position fetched: 72409 -> 10.3390/vaccines8030482
Position fetched: 72410 -> 10.3390/vaccines8030495
Position fetched: 72411 -> 10.3390/vaccines8030508
Position fetched: 72412 -> 10.3

Position fetched: 72526 -> 10.3402/ehtj.v6i0.19944
Position fetched: 72527 -> 10.3402/ehtj.v6i0.19955
Position fetched: 72528 -> 10.3402/gha.v6i0.20100
                                             affiliation  \
71829  [{'affiliation-city': 'Edmonton', 'affilname':...   
71830  [{'affiliation-city': 'Krakow', 'affilname': '...   
71831  [{'affiliation-city': 'Singapore City', 'affil...   
71832  [{'affiliation-city': 'Geneva', 'affilname': '...   
71833  [{'affiliation-city': 'Bern', 'affilname': 'Un...   
...                                                  ...   
72524                                               None   
72525  {'affiliation-city': 'Beijing', 'affilname': '...   
72526  [{'affiliation-city': 'New York', 'affilname':...   
72527  [{'affiliation-city': 'Abu Dis', 'affilname': ...   
72528  [{'affiliation-city': 'Dubai', 'affilname': 'D...   

                                                coredata  
71829  {'srctype': 'j', 'prism:issueIdentifier': '4',...  
71830  {'

Position fetched: 72632 -> 10.3758/s13414-020-02140-w
Position fetched: 72633 -> 10.3758/s13414-020-02142-8
Position fetched: 72634 -> 10.3758/s13414-020-02187-9
Position fetched: 72635 -> 10.3758/s13414-020-02213-w
Position fetched: 72636 -> 10.3758/s13414-020-02220-x
Position fetched: 72637 -> 10.3758/s13421-020-01073-x
Position fetched: 72638 -> 10.3758/s13421-020-01079-5
Position fetched: 72639 -> 10.3758/s13421-020-01117-2
Position fetched: 72640 -> 10.3758/s13421-020-01128-z
Position fetched: 72641 -> 10.3758/s13421-020-01129-y
Position fetched: 72642 -> 10.3758/s13423-020-01767-y
Position fetched: 72643 -> 10.3758/s13423-020-01833-5
Position fetched: 72644 -> 10.3758/s13428-011-0175-8
Position fetched: 72645 -> 10.3758/s13428-020-01446-9
Position fetched: 72646 -> 10.3758/s13428-020-01503-3
Position fetched: 72647 -> 10.3758/s13428-020-01514-0
Position fetched: 72648 -> 10.3758/s13428-020-01515-z
Position fetched: 72649 -> 10.3758/s13428-020-01517-x
Position fetched: 72650 -> 10

Position fetched: 72766 -> 10.3904/kjim.2020.329
Position fetched: 72767 -> 10.3906/biy-2004-113
Position fetched: 72768 -> 10.3906/biy-2005-111
Position fetched: 72769 -> 10.3906/biy-2005-112
Position fetched: 72770 -> 10.3906/biy-2005-114
Position fetched: 72771 -> 10.3906/biy-2005-115
Position fetched: 72772 -> 10.3906/biy-2005-21
Position fetched: 72773 -> 10.3906/biy-2005-35
Position fetched: 72774 -> 10.3906/biy-2005-46
Position fetched: 72775 -> 10.3906/biy-2005-69
Position fetched: 72776 -> 10.3906/biy-2006-1
Position fetched: 72777 -> 10.3906/sag-1807-231
Position fetched: 72778 -> 10.3906/sag-2004-167
Position fetched: 72779 -> 10.3906/sag-2004-174
Position fetched: 72780 -> 10.3906/sag-2004-187
Position fetched: 72781 -> 10.3906/sag-2005-385
Position fetched: 72782 -> 10.3906/sag-2005-395
Position fetched: 72783 -> 10.3906/sag-2006-140
Position fetched: 72784 -> 10.3906/sag-2006-96
Position fetched: 72785 -> 10.3906/sag-2009-140
Position fetched: 72786 -> 10.3906/sag-2009-77

The following cell is useful when the process above is interrupted. Therefore, the dictionary containing fetched information can be narrowed down to useful entries. 

In [ ]:
# def save_new_extra_info(len_df_current_extra_info, upto):
#     """
#     This function is used to separate successfull API calls from API calls which were prevented due to an invalid API-Key.
#     As a result, this function returns a range of valid entries up to the given parameter. 
#     """
#     dict_new_extra_info_saver = dict()
#     i = len_df_current_extra_info
#     while i < upto:
#         #print("Position: " + str(i) + " -> " +  doi_counted.index[i])
#         dict_new_extra_info_saver[i] = dict_new_extra_info[i]
#         i = i + 1 
#     return dict_new_extra_info_saver

The existing and newly fetched information are combined into one DataFrame. 

In [ ]:
# df_combined_extra_info = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info)
# df_combined_extra_info

In [ ]:
#to big for GitHub
#df_combined_extra_info.to_csv('extra_info_CS5099.csv', sep='\t')

Both Dataframes columns are stored each to an object. The series objects are stored to each to a pkl-file which is not exceeding the size of 100MB allowing GitHub uploads.

In [ ]:
# stored_series = store_df_columns(df_combined_extra_info)
# stored_series[0]

In [ ]:
# stored_series[1]

Verifying that the returned None values are due to non existent data and not to an invalid API-Key

In [ ]:
# len_data = len(stored_series[0])
# len_data 

In [ ]:
# ser_doi = pd.Series(doi_counted.index[:len_data])
# ser_doi

In [ ]:
# df_current_extra_info_checker = df_combined_extra_info
# df_current_extra_info_checker['doi'] = ser_doi

In [ ]:
# %%time
# len_df_current_extra_info_checker = len(df_current_extra_info_checker)
# dict_new_extra_info_checker = dict()
# i = 0 
# while i < len_df_current_extra_info_checker: ###################################################### 
#     if df_current_extra_info_checker['affiliation'][i] == None:
#         dict_new_extra_info_checker[i] = fetch_scopus_api(client, ser_doi[i])
#         print("Position fetched again: " + str(i) + " -> " +  ser_doi[i])
#     i = i + 1    

In [ ]:
# dict_new_extra_info_checker
# -> check if at least one value is not None -> otherwise the process is finished here

In [ ]:
# len(dict_new_extra_info_checker)

In [ ]:
# df_combined_extra_info_fetched_again  = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info_checker)
# df_combined_extra_info_fetched_again

In [ ]:
# store_df_columns(df_combined_extra_info_fetched_again)

In [ ]:
# df_combined_extra_info_fetched_again['check_doi'] = ser_doi
# df_combined_extra_info_fetched_again.head(30)